In [1]:
from dask.distributed import Client

# Connect to the Dask scheduler
client = Client('dask-scheduler:8786')

# Check cluster status
print(client)
print(f"Workers: {len(client.scheduler_info()['workers'])}")

<Client: 'tcp://10.42.0.20:8786' processes=0 threads=0, memory=0 B>
Workers: 0


/opt/conda/lib/python3.10/site-packages/distributed/client.py:1391: VersionMismatchWarning: Mismatched versions found

+---------+-----------------+-----------------+---------+
| Package | Client          | Scheduler       | Workers |
+---------+-----------------+-----------------+---------+
| python  | 3.10.13.final.0 | 3.10.12.final.0 | None    |
+---------+-----------------+-----------------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


We are loading in the HIGGS dataset from https://archive.ics.uci.edu/dataset/280/higgs

The features with an `m_` preprended to it are pre-engineered features used to aid in classification

In [2]:
import dask.dataframe as dd

columns = [
    "class_label",
    "lepton_pT", "lepton_eta", "lepton_phi",
    "missing_energy_magnitude", "missing_energy_phi",
    "jet1_pt", "jet1_eta", "jet1_phi", "jet1_btag",
    "jet2_pt", "jet2_eta", "jet2_phi", "jet2_btag",
    "jet3_pt", "jet3_eta", "jet3_phi", "jet3_btag",
    "jet4_pt", "jet4_eta", "jet4_phi", "jet4_btag",
    "m_jj", "m_jjj", "m_lv", "m_jlv", "m_bb", "m_wbb", "m_wwbb"
]

df = dd.read_csv('/data/HIGGS.csv', header=None, names=columns)

In [22]:
df.head()

,class_label,lepton_pT,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet1_pt,jet1_eta,jet1_phi,jet1_btag,...,jet4_eta,jet4_phi,jet4_btag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
0,1.0,0.869293,-0.635082,0.225690,0.327470,-0.689993,0.754202,-0.248573,-1.092064,0.000000,...,-0.010455,-0.045767,3.101961,1.353760,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,1.0,0.907542,0.329147,0.359412,1.497970,-0.313010,1.095531,-0.557525,-1.588230,2.173076,...,-1.138930,-0.000819,0.000000,0.302220,0.833048,0.985700,0.978098,0.779732,0.992356,0.798343
2,1.0,0.798835,1.470639,-1.635975,0.453773,0.425629,1.104875,1.282322,1.381664,0.000000,...,1.128848,0.900461,0.000000,0.909753,1.108330,0.985692,0.951331,0.803252,0.865924,0.780118
3,0.0,1.344385,-0.876626,0.935913,1.992050,0.882454,1.786066,-1.646778,-0.942383,0.000000,...,-0.678379,-1.360356,0.000000,0.946652,1.028704,0.998656,0.728281,0.869200,1.026736,0.957904
4,1.0,1.105009,0.321356,1.522401,0.882808,-1.205349,0.681466,-1.070464,-0.921871,0.000000,...,-0.373566,0.113041,0.000000,0.755856,1.361057,0.986610,0.838085,1.133295,0.872245,0.808487


In [23]:
row_count = df.shape[0].compute()
row_count

11000000

In [25]:
df.describe().compute()

,class_label,lepton_pT,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet1_pt,jet1_eta,jet1_phi,jet1_btag,...,jet4_eta,jet4_phi,jet4_btag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
count,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,...,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07
mean,5.299203e-01,9.914658e-01,-8.297618e-06,-1.327225e-05,9.985364e-01,2.613459e-05,9.909152e-01,-2.027520e-05,7.716199e-06,9.999687e-01,...,-5.756954e-06,1.744903e-05,1.000000e+00,1.034290e+00,1.024805e+00,1.050554e+00,1.009742e+00,9.729596e-01,1.033036e+00,9.598120e-01
std,4.991040e-01,5.653777e-01,1.008827e+00,1.006346e+00,6.000185e-01,1.006326e+00,4.749747e-01,1.009303e+00,1.005901e+00,1.027808e+00,...,1.007694e+00,1.006366e+00,1.400209e+00,6.746354e-01,3.808074e-01,1.645763e-01,3.974453e-01,5.254063e-01,3.652556e-01,3.133378e-01
min,0.000000e+00,2.746966e-01,-2.434976e+00,-1.742508e+00,2.370088e-04,-1.743944e+00,1.375024e-01,-2.969725e+00,-1.741237e+00,0.000000e+00,...,-2.497265e+00,-1.742691e+00,0.000000e+00,7.507046e-02,1.986757e-01,8.304866e-02,1.320062e-01,4.786215e-02,2.951122e-01,3.307214e-01
25%,0.000000e+00,5.938645e-01,-7.266349e-01,-8.547301e-01,5.810280e-01,-8.568112e-01,6.833898e-01,-6.743720e-01,-8.581175e-01,0.000000e+00,...,-7.041963e-01,-8.592708e-01,0.000000e+00,7.927191e-01,8.480743e-01,9.858175e-01,7.703856e-01,6.782121e-01,8.214605e-01,7.723097e-01
50%,1.000000e+00,8.590447e-01,1.260744e-02,1.928187e-02,8.976148e-01,1.716515e-02,8.982088e-01,1.383778e-02,1.291151e-02,1.086538e+00,...,1.286448e-02,1.482118e-02,0.000000e+00,8.965645e-01,9.524049e-01,9.898756e-01,9.190750e-01,8.771406e-01,9.493063e-01,8.741697e-01
75%,1.000000e+00,1.244828e+00,7.508758e-01,8.826461e-01,1.301897e+00,8.818105e-01,1.176695e+00,7.010574e-01,8.816177e-01,2.173076e+00,...,7.240956e-01,8.849233e-01,3.101961e+00,1.028994e+00,1.086214e+00,1.024106e+00,1.146263e+00,1.145739e+00,1.146450e+00,1.064168e+00
max,1.000000e+00,1.209891e+01,2.434868e+00,1.743236e+00,1.539682e+01,1.743257e+00,9.940391e+00,2.969674e+00,1.741454e+00,2.173076e+00,...,2.498009e+00,1.743372e+00,3.101961e+00,4.019237e+01,2.037278e+01,7.992739e+00,1.426244e+01,1.776285e+01,1.149652e+01,8.374498e+00


11M rows and at a glance it looks like everything is normalized.

In [7]:
df.to_parquet("/data/HIGGS.parquet")

In [3]:
import dask.dataframe as dd

df = dd.read_parquet("/data/HIGGS.parquet")

In [ ]:
df.head()

In [10]:
df.shape[0].compute()

11000000

Oh wow that is so much faster as a parquet dataframe

In [11]:
df.describe().compute()

,class_label,lepton_pT,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet1_pt,jet1_eta,jet1_phi,jet1_btag,...,jet4_eta,jet4_phi,jet4_btag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
count,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,...,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07
mean,5.299203e-01,9.914658e-01,-8.297618e-06,-1.327225e-05,9.985364e-01,2.613459e-05,9.909152e-01,-2.027520e-05,7.716199e-06,9.999687e-01,...,-5.756954e-06,1.744903e-05,1.000000e+00,1.034290e+00,1.024805e+00,1.050554e+00,1.009742e+00,9.729596e-01,1.033036e+00,9.598120e-01
std,4.991040e-01,5.653777e-01,1.008827e+00,1.006346e+00,6.000185e-01,1.006326e+00,4.749747e-01,1.009303e+00,1.005901e+00,1.027808e+00,...,1.007694e+00,1.006366e+00,1.400209e+00,6.746354e-01,3.808074e-01,1.645763e-01,3.974453e-01,5.254063e-01,3.652556e-01,3.133378e-01
min,0.000000e+00,2.746966e-01,-2.434976e+00,-1.742508e+00,2.370088e-04,-1.743944e+00,1.375024e-01,-2.969725e+00,-1.741237e+00,0.000000e+00,...,-2.497265e+00,-1.742691e+00,0.000000e+00,7.507046e-02,1.986757e-01,8.304866e-02,1.320062e-01,4.786215e-02,2.951122e-01,3.307214e-01
25%,0.000000e+00,5.938645e-01,-7.266349e-01,-8.547301e-01,5.810280e-01,-8.568112e-01,6.833898e-01,-6.743720e-01,-8.581175e-01,0.000000e+00,...,-7.041963e-01,-8.592708e-01,0.000000e+00,7.927191e-01,8.480743e-01,9.858175e-01,7.703856e-01,6.782121e-01,8.214605e-01,7.723097e-01
50%,1.000000e+00,8.590447e-01,1.260744e-02,1.928187e-02,8.976148e-01,1.716515e-02,8.982088e-01,1.383778e-02,1.291151e-02,1.086538e+00,...,1.286448e-02,1.482118e-02,0.000000e+00,8.965645e-01,9.524049e-01,9.898756e-01,9.190750e-01,8.771406e-01,9.493063e-01,8.741697e-01
75%,1.000000e+00,1.244828e+00,7.508758e-01,8.826461e-01,1.301897e+00,8.818105e-01,1.176695e+00,7.010574e-01,8.816177e-01,2.173076e+00,...,7.240956e-01,8.849233e-01,3.101961e+00,1.028994e+00,1.086214e+00,1.024106e+00,1.146263e+00,1.145739e+00,1.146450e+00,1.064168e+00
max,1.000000e+00,1.209891e+01,2.434868e+00,1.743236e+00,1.539682e+01,1.743257e+00,9.940391e+00,2.969674e+00,1.741454e+00,2.173076e+00,...,2.498009e+00,1.743372e+00,3.101961e+00,4.019237e+01,2.037278e+01,7.992739e+00,1.426244e+01,1.776285e+01,1.149652e+01,8.374498e+00
